In [ ]:
import pygame
import sys
from main import build_player_adjacency, resolve_name_to_ids, bfs_shortest_path, annotate_path_edges
from collections import defaultdict

data_file = "nba_rosters_1999_to_present_working.json"

graph = build_player_adjacency(data_file)
all_names = sorted(list(graph.keys()))
id2name = {i: name for i, name in enumerate(all_names)}
name2id = {name: i for i, name in enumerate(all_names)}
name2ids = defaultdict(set)
for name, i in name2id.items():
    name2ids[name.lower()].add(i)
graph = {name2id[name]: {name2id[n] for n in neighbors} for name, neighbors in graph.items()}

pygame.init()

W, H = 1000, 680
screen = pygame.display.set_mode((W, H))
pygame.display.set_caption("NBA Teammate Path - You vs Computer (BFS)")

font = pygame.font.Font(None, 28)
big = pygame.font.Font(None, 36)

# simple text input helper
class TextBox:
    def __init__(self, x, y, w, h, placeholder=""):
        self.rect = pygame.Rect(x, y, w, h)
        self.text = ""
        self.placeholder = placeholder
        self.active = False
    def handle(self, ev):
        if ev.type == pygame.MOUSEBUTTONDOWN:
            self.active = self.rect.collidepoint(ev.pos)
        if self.active and ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_RETURN: 
                self.active = False
            elif ev.key == pygame.K_BACKSPACE:
                self.text = self.text[:-1]
            else:
                self.text += ev.unicode
    def draw(self, surf):
        pygame.draw.rect(surf, (230,230,230), self.rect, border_radius=8)
        txt = self.text if self.text else self.placeholder
        color = (0,0,0) if self.text else (120,120,120)
        surf.blit(font.render(txt, True, color), (self.rect.x+8, self.rect.y+7))
        pygame.draw.rect(surf, (40,40,40), self.rect, 2, border_radius=8)

src_box = TextBox(30, 30, 420, 40, "Start player (e.g., 'LeBron James')")
dst_box = TextBox(30, 80, 420, 40, "Target player (e.g., 'Kobe Bryant')")

# user path state
user_path_ids = []
computer_path_ids = []
status_msg = ""

def name_or_id_to_id(token):
    # allow id entry too
    token = token.strip()
    if token.isdigit():
        return int(token) if int(token) in id2name else None
    matches = list(resolve_name_to_ids(token, name2ids))
    if len(matches) == 1:
        return matches[0]
    return None  # not found

def format_path(ids):
    if not ids: return "—"
    return "  →  ".join(id2name[i] for i in ids)

def compute_bfs():
    global computer_path_ids, status_msg
    s = name_or_id_to_id(src_box.text)
    t = name_or_id_to_id(dst_box.text)
    if s is None or t is None:
        status_msg = "Could not find. Try full names."
        computer_path_ids = []
        return
    computer_path_ids = bfs_shortest_path(graph, s, t)
    if not computer_path_ids:
        status_msg = "No connection buddy."
    else:
        L = len(computer_path_ids)-1
        status_msg = f"Computer shortest path length: {L}"

def reset_user():
    global user_path_ids, status_msg
    user_path_ids = []
    status_msg = "the user path cleared."

def push_user_next(name):
    global user_path_ids, status_msg
    pid = name_or_id_to_id(name)
    if pid is None:
        status_msg = "Name not recognized"
        return
    if not user_path_ids:
        user_path_ids.append(pid)
        status_msg = f"Started at {id2name[pid]}"
        return
    #  teammate adjacency
    if pid in graph.get(user_path_ids[-1], []):
        user_path_ids.append(pid)
        status_msg = f"Added {id2name[pid]}"
    else:
        status_msg = f"{id2name[user_path_ids[-1]]} and {id2name[pid]} were never teammates."

#  ui buttons
def draw_button(rect, label):
    pygame.draw.rect(screen, (70,140,255), rect, border_radius=10)
    pygame.draw.rect(screen, (20,40,80), rect, 2, border_radius=10)
    txt = font.render(label, True, (255,255,255))
    screen.blit(txt, (rect.x + (rect.w - txt.get_width())//2, rect.y + (rect.h - txt.get_height())//2))

compute_btn = pygame.Rect(470, 30, 160, 40)
reset_btn   = pygame.Rect(470, 80, 160, 40)
add_user_box = TextBox(30, 150, 420, 40, "Type next teammate in YOUR path")
add_btn = pygame.Rect(470, 150, 160, 40)

instructions_font = pygame.font.Font(None, 24)
instructions_text = [
    "instructions:",
    "enter a start and target player.",
    "click 'compute bfs' to see the shortest path.",
    "build your own path by adding teammates.",
    "click 'reset user' to clear your path."
]

clock = pygame.time.Clock()

while True:
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        src_box.handle(ev); dst_box.handle(ev); add_user_box.handle(ev)

        if ev.type == pygame.MOUSEBUTTONDOWN:
            if compute_btn.collidepoint(ev.pos):
                compute_bfs()
            if reset_btn.collidepoint(ev.pos):
                reset_user()
            if add_btn.collidepoint(ev.pos):
                if add_user_box.text.strip():
                    push_user_next(add_user_box.text.strip())
                    add_user_box.text = ""

    screen.fill((250, 250, 255))

    # inputs
    src_box.draw(screen); dst_box.draw(screen)
    add_user_box.draw(screen)
    draw_button(compute_btn, "Compute with BFS")
    draw_button(reset_btn, "Reset User")
    draw_button(add_btn, "Add Teammate")

    # instructions
    for i, line in enumerate(instructions_text):
        instructions_surface = instructions_font.render(line, True, (0, 0, 0))
        screen.blit(instructions_surface, (650, 30 + i * 30))

    # headings
    screen.blit(big.render("Your Path", True, (0,0,0)), (30, 220))
    screen.blit(big.render("Computer Path (Shortest)", True, (0,0,0)), (30, 400))

    # render paths
    user_text = format_path(user_path_ids)
    comp_text = format_path(computer_path_ids)
    screen.blit(font.render(user_text, True, (30,30,30)), (30, 260))
    screen.blit(font.render(comp_text, True, (30,30,30)), (30, 440))

    
    screen.blit(font.render(status_msg, True, (180,0,0)), (30, 620))

    pygame.display.flip()
    clock.tick(60)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
